In [2]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

def create_fuzzy_system():
    temperature = ctrl.Antecedent(np.arange(-40, 41, 1), 'temperature')
    humidity = ctrl.Antecedent(np.arange(0, 101, 1), 'humidity')
    motor_speed = ctrl.Consequent(np.arange(0, 21, 1), 'motor_speed')

    temperature['low'] = fuzz.trimf(temperature.universe, [-40, -40, 0])
    temperature['moderate'] = fuzz.trimf(temperature.universe, [-10, 0, 30])
    temperature['high'] = fuzz.trimf(temperature.universe, [20, 40, 40])

    humidity['low'] = fuzz.trimf(humidity.universe, [0, 0, 40])
    humidity['moderate'] = fuzz.trimf(humidity.universe, [30, 50, 70])
    humidity['high'] = fuzz.trimf(humidity.universe, [60, 100, 100])

    motor_speed['low'] = fuzz.trimf(motor_speed.universe, [0, 0, 10])
    motor_speed['high'] = fuzz.trimf(motor_speed.universe, [10, 20, 20])

    rule1 = ctrl.Rule(temperature['high'] & humidity['low'], motor_speed['low'])
    rule2 = ctrl.Rule(temperature['moderate'] & humidity['low'], motor_speed['low'])
    rule3 = ctrl.Rule(temperature['low'] & humidity['low'], motor_speed['high'])
    rule4 = ctrl.Rule(humidity['moderate'], motor_speed['high'])
    rule5 = ctrl.Rule(temperature['low'] & humidity['high'], motor_speed['high'])
    rule6 = ctrl.Rule(humidity['high'], motor_speed['high'])

    control_system = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6])
    return control_system

def get_motor_speed(control_system, temp, hum):
    simulator = ctrl.ControlSystemSimulation(control_system)
    simulator.input['temperature'] = temp
    simulator.input['humidity'] = hum
    simulator.compute()
    return simulator.output['motor_speed']

control_system = create_fuzzy_system()

print("\ntest 1:")
temp, hum = 20, 90
print(f"Temperature: {temp}°C, Humidity: {hum}%")
motor_speed = get_motor_speed(control_system, temp, hum)
print(f"Motor Speed: {motor_speed:.2f}")

test_cases = [
    (-30, 20),
    (0, 50),
    (35, 30),
]

for i, (temp, hum) in enumerate(test_cases, 2):
    print(f"\ntest {i}:")
    print(f"Temperature: {temp}°C, Humidity: {hum}%")
    motor_speed = get_motor_speed(control_system, temp, hum)
    print(f"Motor Speed: {motor_speed:.2f}")


test 1:
Temperature: 20°C, Humidity: 90%
Motor Speed: 16.50

test 2:
Temperature: -30°C, Humidity: 20%
Motor Speed: 16.11

test 3:
Temperature: 0°C, Humidity: 50%
Motor Speed: 16.67

test 4:
Temperature: 35°C, Humidity: 30%
Motor Speed: 4.40
